In [3]:
import pandas as pd
import re
from cleaning_scripts.load_data_to_clean2 import load_data_to_clean
from cleaning_scripts.string_operations import create_artist_name_col

ImportError: cannot import name 'create_artist_name_col' from 'cleaning_scripts.string_operations' (/Users/CUDAN/Documents/GitHub/data_analysis_portfolio/cleaning_scripts/string_operations.py)

In [2]:
with open('cleaning_scripts/string_operations.py', 'r') as file:
    print(file.read())

In [23]:
import pandas as pd

# Sample DataFrame to test the function
df_test = pd.DataFrame({
    'artist_name': ['Artist A', 'Artist B'],
    'object_title': ['Artwork 1', 'Artwork 2']
})

# Call the function
df_test = create_artist_name(df_test)

# Check the result
print(df_test)


NameError: name 'create_artist_name' is not defined

In [13]:
raw_data, dataset_names = load_data_to_clean()

Loaded met with shape: (476318, 54)
Loaded reina_sofia with shape: (10018, 16)
Loaded tate with shape: (69201, 31)
Loaded pompidou with shape: (115217, 23)
Loaded moma with shape: (157474, 30)
Loaded whitney with shape: (4012, 8)
Loaded national_gallery with shape: (141438, 55)
Loaded flattened data with shape: (58264, 499)


KeyboardInterrupt: 

create functions for cleaning fields.
artist name, artwork title, artwork medium, and acquisition source, aquisition year, creation year, artist nationality, artist gender

In [12]:
transformed_data = []
for df in raw_data:
    create_artist_name(df)
    transformed_data.append(df)

NameError: name 'raw_data' is not defined

In [3]:
def create_artist_name(df):
    # List of possible column names for the artist name
    possible_columns = ['artist', 'author', 'artist_name', 'Artist Display Name', 'artist name', 'Artist Display Name', 'Artist', 'artists', 
                        'forwarddisplayname', 'production_0_creator']
    
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Artist'] = df[col]
            break  # Exit loop once we find the first match
    
    return df

In [4]:
def create_artwork_title(df):
    # List of possible column names for the artist name
    possible_columns = ['object_title', 'artwork_name', 'Title', 'url', 'name', 'title', 'title_fi', 'titles_0_title']
    
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Title'] = df[col]
            break  # Exit loop once we find the first match
    
    return df

In [5]:
from extract_first_number import extract_first_number

    
def clean_acquisition_year(df):
   # List of possible column names for the artist name
    possible_columns = ['acquisition_date', 'credit_line', 'AccessionYear', 'artwork_acquisition', 'year_adquisition', 'acquisitionYear',
                        'DateAcquired', 'accession_number', 'accessionnum', 'inventoryNumber', 'acquisition_date_precision', 'AcquiredDate' ]
    df['Year_acquisition'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Year_acquisition'] = df[col].apply(extract_first_number)
            break  # Exit loop once we find the first match
    
    return df 

In [7]:
#I need to add more tags based on dataset values
from medium_tags import load_medium_tags #returns medium_tags, medium_name
medium_tags, medium_name = load_medium_tags()
def classify_medium(df):
    df['Medium_classified'] = pd.NA
    
     # Iterate over each row in the DataFrame
    for idx, row in df.iterrows():
        medium = row['Medium_raw'] 
        
        # Skip if the 'Medium' value is not a string (e.g., NaN or other types)
        if not isinstance(medium, str):
            continue
        
        # Check each tag group (e.g., ['paper', 'watercolor'], ['oil'])
        for i, tag_group in enumerate(medium_tags):
            # Check if any keyword from the tag group exists in the 'Medium' column
            if any(tag.lower() in medium.lower() for tag in tag_group):
                # Assign the corresponding medium name (from medium_name list)
                df.at[idx, 'Medium_classified'] = medium_name[i]
                break  # Stop once we find a match
    
    return df

In [8]:
#i need to update the dictionary so all nationalities will be translated into countries
from load_nationality_tags import load_nationality_tags
nationality_tags, country_name = load_nationality_tags()
def create_artist_nationality(df):
    possible_columns = ['Country','Artist Nationality', 'nationality_artist', 'artist_nationality', 'Nationality', 'nationality', 
                        'production_0_creator_nationality']
    df['Country_calculated'] = pd.NA
    df['nationality_raw'] = pd.NA

    # Check each possible column for nationality information
    for col in possible_columns:
        if col in df.columns:
            df['nationality_raw'] = df[col].str.lower().str.strip('the').str.replace('(', '').str.replace(')', '').str.split(' ').str[0].str.strip(',')  # Copy the nationality info into 'nationality_raw'
            break  # Exit loop once we find the first match 

    for idx, row in df.iterrows():
        nationality = row['nationality_raw'] 
        
        # Skip if the 'nationality' value is not a string (e.g., NaN or other types)
        if not isinstance(nationality, str):
            continue
        
        # Check each tag group (e.g., ['paper', 'watercolor'], ['oil'])
        for i, tag_group in enumerate(nationality_tags):
            # Check if any keyword from the tag group exists in the 'nationality' column
            if any(tag.lower() in nationality.lower() for tag in tag_group):
                # Assign the corresponding nationality name (from nationality_name list)
                df.at[idx, 'Country_calculated'] = country_name[i]
                break  # Stop once we find a match

    return df


use functions for raw datasets and dreate a new list with clean datasets

In [9]:
def create_artist_gender(df):
    possible_columns = ['Artist Gender', 'gender', 'artist_gender', 'Gender', 'production_0_creator_gender']
    # Define a gender dictionary to map common terms to 'female' or 'male'
    gender_dict = {'female': ["female", 'woman'], 'male': ['male', 'man']}
    
    # Initialize the 'Gender_raw' column with missing values
    df['Gender_raw'] = pd.NA
    
    # Check which column exists and assign to 'Gender_raw'
    for col in possible_columns:
        if col in df.columns:
            df['Gender_raw'] = df[col]
            break  # Exit loop once we find the first match
    
    # Clean the text: remove all non-alphanumeric characters, split by spaces, and use the first word
    df['Gender_classified'] = df['Gender_raw'].str.replace(r'[^a-zA-Z\s]', '', regex=True)  # Remove non-alphabetic characters
    df['Gender_classified'] = df['Gender_classified'].str.split().str[0].str.lower()  # Split by space and take the first word

    # Manually map "man" and "woman" to "male" and "female" directly
    df['Gender_classified'] = df['Gender_classified'].replace({'man': 'male', 'woman': 'female'})
    
    # Use a safe check to apply gender classification only on valid strings
    df['Gender_classified'] = df['Gender_classified'].apply(
        lambda x: 'female' if isinstance(x, str) and 'female' in x else ('male' if isinstance(x, str) and 'male' in x else x)
    )
    
    # Return the modified DataFrame
    return df


In [14]:
test = transformed_data[10][['acquisition_raw', 'Acquistion_classified']].drop_duplicates(subset=['acquisition_raw'])



In [10]:
from load_acquisition_tags import load_acquisition_tags
acqusition_tags, acquisition_methods = load_acquisition_tags()
def create_acquisition_method(df):
    possible_columns = ['Credit Line', 'creditLine', 'acquisition_type', 'CreditLine', 'credit_line', 'creditline', 'CreditLine']
     # Initialize the 'Gender_raw' column with missing values
    df['acquisition_raw'] = pd.NA
    
    # Check which column exists and assign to 'Gender_raw'
    for col in possible_columns:
        if col in df.columns:
            df['acquisition_raw'] = df[col].str.lower()
            break  # Exit loop once we find the first match

    for idx, row in df.iterrows():
        acquisition = row['acquisition_raw'] 
        
        # Skip if the 'acquisition' value is not a string (e.g., NaN or other types)
        if not isinstance(acquisition, str):
            continue
        
        # Check each tag group 
        for i, tag_group in enumerate(acqusition_tags):
            # Check if any keyword from the tag group exists in the 'acquisition' column
            if any(tag.lower() in acquisition.lower() for tag in tag_group):
                # Assign the corresponding acquisition name (from acquisition_name list)
                df.at[idx, 'Acquistion_classified'] = acquisition_methods[i]
                break  # Stop once we find a match
    return df
    

In [ ]:
#acquisition date

In [11]:
def artwork_creation_date(df):
    possible_columns = ['Object Date', 'year_production', 'year', 'object_date', 'Date', 'display_date', 'endyear_x', 'yearFrom', 
                        'production_date_0_end', 'DateCreated']
    
    df['Date_creation_year'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Date_creation_year'] = df[col].apply(extract_first_number)
            break  # Exit loop once we find the first match
    
    return df 

In [12]:
def artist_birth_year(df):
    possible_columns = ['Artist Begin Date', 'author_born_year', 'yearOfBirth', 'artist_birth', 'BeginDate', 'birth_date', 
                        'beginyear', 'people_0_birthDate', 'production_0_creator_date_of_birth']
    df['Artist_birth_year'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Artist_birth_year'] = df[col].apply(extract_first_number)
            break  # Exit loop once we find the first match
    
    return df 
    

In [13]:
def artist_death_year(df):
    possible_columns = ['Artist End Date', 'author_death_year', 'yearOfDeath', 'artist_death', 'EndDate', 'death_date', 'endyear', 
                        'people_0_deathDate', 'production_0_creator_date_of_death']
    df['Artist_death_year'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Artist_death_year'] = df[col].apply(extract_first_number)
            break  # Exit loop once we find the first match
    
    return df

In [14]:
transformed_data = []
for df in raw_data:
    create_artist_name(df)
    create_artwork_title(df)
    create_artwork_medium(df)
    classify_medium(df)
    create_artist_nationality(df)
    artwork_creation_date(df)
    clean_acquisition_year(df)
    artist_birth_year(df)
    artist_death_year(df)
    create_artist_gender(df)
    create_acquisition_method(df)
    transformed_data.append(df)


In [15]:
transformed_data[0].columns

Index(['Object Number', 'Is Highlight', 'Is Timeline Work', 'Is Public Domain',
       'Object ID', 'Gallery Number', 'Department', 'AccessionYear',
       'Object Name', 'Title', 'Culture', 'Period', 'Dynasty', 'Reign',
       'Portfolio', 'Constituent ID', 'Artist Role', 'Artist Prefix',
       'Artist Display Name', 'Artist Display Bio', 'Artist Suffix',
       'Artist Alpha Sort', 'Artist Nationality', 'Artist Begin Date',
       'Artist End Date', 'Artist Gender', 'Artist ULAN URL',
       'Artist Wikidata URL', 'Object Date', 'Object Begin Date',
       'Object End Date', 'Medium', 'Dimensions', 'Credit Line',
       'Geography Type', 'City', 'State', 'County', 'Country', 'Region',
       'Subregion', 'Locale', 'Locus', 'Excavation', 'River', 'Classification',
       'Rights and Reproduction', 'Link Resource', 'Object Wikidata URL',
       'Metadata Date', 'Repository', 'Tags', 'Tags AAT URL',
       'Tags Wikidata URL', 'Artist', 'Medium_raw', 'Medium_classified',
       'Countr

save clean datasets in separate csv files

In [17]:
import pandas as pd

clean_data = []

# Define the desired columns
required_columns = ['Artist', 'Title', 'Medium', 'Medium_classified', 'Acquistion_classified', 
                    'Year_acquisition', 'Gender_classified', 'Artist_birth_year', 
                    'Artist_death_year', 'Country_calculated', 'Date_creation_year']

for df in transformed_data:
    # Add any missing columns as NaN
    for col in required_columns:
        if col not in df.columns:
            df[col] = pd.NA  # or use 'None' or 'NaN' depending on your preference

    clean_data.append(df[required_columns])



In [18]:

# Save each dataset using the corresponding name
for dataset, name in zip(clean_data, dataset_names):
    # Generate a unique filename using the dataset name
    file_name = f"{name}.csv"
    
    # Save the DataFrame to a CSV file
    dataset.to_csv(file_name, index=False)
    print(f"Saved {file_name}")

Saved met.csv
Saved reina_sofia.csv
Saved tate.csv
Saved pompidou.csv
Saved moma.csv
Saved whitney.csv
Saved national_gallery.csv
Saved kiasma.csv
Saved smk.csv
Saved ateneum.csv
Saved queensland.csv
